In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import traja

## Load Data

f_name, 0 
food_type, 1 
starve_period, 2 
xcm, 3 
xcm_dot,
ycm,
ycm_dot,
num_meals, 7 
tot_vol,
dur_eating, 
eat_latency, 10
meal_num,
start_time, 12
end_time, 13
meal_dur,
meal_vol, 15
t, 16

In [2]:
data = np.load('newrawdata.npy',allow_pickle=True)
flynum = data[1].shape[0] #384

In [3]:
food_type = data[1]
starve_period = data[2]

xcm = data[3] #x center of mass location
ycm = data[5]
end_time = data[13]

meal_vol = data[15]

t_offset = 1.0/3.0 
t = data[16] - t_offset

## Trajectory Calculations

In [4]:
# ? seconds before 1st meal -> 1st meal
first_meal_end = np.array([end_time[i][0] for i in range(flynum)])

# get that trajectory
x_traj = []
y_traj = []
for i in range(flynum):
    idx = (t[i] >= (first_meal_end[i])) & (t[i] <= (first_meal_end[i] + 5))   #5 seconds after 1st meal
    x_traj.append(xcm[i][idx])
    y_traj.append(ycm[i][idx])
x_traj = np.array(x_traj,dtype=object)
y_traj = np.array(y_traj,dtype=object)

def traj(flyno):
    return np.vstack((x_traj[flyno], y_traj[flyno]))

from traja import TrajaCollection
def create_df(flyno):
    x_fly = traj(flyno)[0]
    y_fly = traj(flyno)[1]
    d = {'x': x_fly, 'y': y_fly}
    df = pd.DataFrame(data=d)
    df = traja.TrajaDataFrame(df)
    return df

# Create a dictionary of DataFrames, with 'id' as key.
keys = np.arange(flynum)
all_values = [] #create values - i.e. df for each fly
for n in range(flynum):
    all_values.append(create_df(n))

df_all = {}
for i in range(flynum):
    df_all[i] = all_values[i]

# Create a TrajaCollection df (x,y,id)
trjs = TrajaCollection(df_all)

## Finding Conditions

In [5]:
#By Foodtype
mM1000 = np.where(food_type == '1000mM')[0] #(90,)
mM100 = np.where(food_type == '100mM')[0]
mM10 = np.where(food_type == '10mM')[0]
mM1 = np.where(food_type == '1mM')[0]  #(48,)
mMwater = np.where(food_type == 'water')[0]
all_foodtype = [mM1000, mM100, mM10, mM1, mMwater]

#By starvetime
st_0 = np.where(starve_period == '0')[0]
st_8 = np.where(starve_period == '8')[0]
st_16 = np.where(starve_period == '16')[0]
st_24 = np.where(starve_period == '24')[0]
st_32 = np.where(starve_period == '32')[0]
all_starvedtime = [st_0,st_8,st_16,st_24,st_32]

#Volume of first bout 
vol_first = np.array([meal_vol[i][0] for i in range(flynum)])

In [6]:
#Overlapping Conditions by starved time

allcondition_st_0 = [np.intersect1d(st_0,i) for i in all_foodtype]
allcondition_st_8 = [np.intersect1d(st_8,i) for i in all_foodtype]
allcondition_st_16 = [np.intersect1d(st_16,i) for i in all_foodtype]
allcondition_st_24 = [np.intersect1d(st_24,i) for i in all_foodtype]
allcondition_st_32 = [np.intersect1d(st_32,i) for i in all_foodtype]

In [7]:
#Overlapping Conditions by foodtype

allcondition_1000 = [np.intersect1d(mM1000,i) for i in all_starvedtime]
allcondition_100 = [np.intersect1d(mM100,i) for i in all_starvedtime]
allcondition_10 = [np.intersect1d(mM10,i) for i in all_starvedtime]
allcondition_1 = [np.intersect1d(mM1,i) for i in all_starvedtime]
allcondition_water = [np.intersect1d(mMwater,i) for i in all_starvedtime]

## Defining 3 states 5 seconds after 1st meal
1.CW ; 2. CCW; 3. stop

In [8]:
def cal_angle_5s(flynum):
    angle = traja.calc_angle(trjs[trjs['id']==flynum])
    turns = np.diff(angle) #diff in angle
    turns[0]=0
    
    fivturns = []
    for i in range(4):
        fivturns.append(turns[(i+1)*30-1] - turns[i*30-1])
    fivturns.append(turns[148] - turns[119])
    fivturns = np.array(fivturns)
    fivturns[0] = turns[29] - turns[0]                                  
    return fivturns


In [9]:
def cw_ccw(flynum):
    angle_5s = cal_angle_5s(flynum)
    two_states = np.zeros(len(angle_5s))
    for i, v in enumerate(angle_5s):
        if v <0: 
            two_states[i] = 1 #CW
        elif v >0:
            two_states[i] = 2 #CCW 
    return two_states

In [10]:
fly_tra = traj(0)
import math
def calculate_distance(starting_x, starting_y, destination_x, destination_y):
    distance = math.hypot(destination_x - starting_x, 
                          destination_y - starting_y)  

    return np.abs(distance)
def velocity(flynum):
    fly_tra = traj(flynum)
    x, y = fly_tra
    velo = [0]
    for i in range(len(x)-1):
        d = calculate_distance(x[i], y[i], x[i+1], y[i+1])
        v = d/1
        velo.append(v)
    return np.array(velo)

def fiv_velocity(flynum):
    v = velocity(flynum)
    fiv_vel = np.array([np.mean(v[0:29]), np.mean(v[29:59]), 
                    np.mean(v[59:89]), np.mean(v[89:119])
                    , np.mean(v[119:149])])
    return fiv_vel

In [11]:
def threestates(flynum):
    two = cw_ccw(flynum)
    v = fiv_velocity(flynum)
    two[v<0.0015]=0   #threshold were made after plotting average velocity
    return two

## An Example of How States Could be Like

In [12]:
for i in allcondition_st_8[1][0:4]:
    print(threestates(i))

[1. 2. 1. 2. 2.]
[0. 0. 0. 1. 0.]
[2. 1. 1. 2. 1.]
[1. 2. 1. 2. 1.]


## Entropy Calculation (information theory)

In [13]:
from scipy.stats import entropy
def entropy_cal(labels, base=None):
    value,counts = np.unique(labels, return_counts=True)
    return entropy(counts, base=base)

def avg_entropy(condition): 
    en = 0
    for i in condition[0]:
        en +=entropy_cal(threestates(i), base=None)
    en = en/len(condition[0])    
    return en

In [14]:
print('Avg Entropy Grouped by Foodtype: ')
fdtype = ['1000mM: ', '100mM: ','10mM: ','1mM: ','water: ']
allfoodtypecond = [allcondition_1000, allcondition_100, allcondition_10,
                  allcondition_1,allcondition_water]
for i in range(len(allfoodtypecond)):
    print(fdtype[i] + str(avg_entropy(allfoodtypecond[i])))

Avg Entropy Grouped by Foodtype: 
1000mM: 0.3167568464110782
100mM: 0.3167568464110782
10mM: 0.0
1mM: 1.0549201679861442
water: 0.47513526961661734


In [15]:
print('Avg Entropy Grouped by Starved Time: ')
sttype = ['0hr: ', '8hr: ','16hr: ','24hr: ','32hr: ']
allstcond = [allcondition_st_0, allcondition_st_8, allcondition_st_16,
                  allcondition_st_24,allcondition_st_32]
for i in range(len(allstcond)):
    print(sttype[i] + str(avg_entropy(allstcond[i])))

Avg Entropy Grouped by Starved Time: 
0hr: 0.3167568464110782
8hr: 0.6110239532181061
16hr: 0.3609269483722018
24hr: 0.39454430293470355
32hr: 0.3590584941838202
